In [16]:
import os
import glob
import json
import base64
import csv
from openai import OpenAI 

# put your OpenAI API key in the environment variable OPENAI_API_KEY
api_key= os.getenv("OPENAI_API_KEY")


In [18]:
# 1. Configuration:  
API_MODEL       = "gpt-4.1-mini"  # or "gpt-4o" if available
OPENAI_API_KEY  = api_key # or os.getenv("OPENAI_API_KEY")

# Instantiate the new client
client = OpenAI(api_key=OPENAI_API_KEY)

# 2. Paths (adjust if needed)
PROMPT_FILE             = "prompt_with_floormap.txt"
NODE_METADATA_FILE      = "node_image_results.json"
FLOORPLAN_IMAGE_PATH    = os.path.join("CEPSR - Floor 7", "Floorplan", "labeled_grid.png")
QUERY_IMAGES_FOLDER     = os.path.join("CEPSR - Floor 7", "Query images")
OUTPUT_CSV              = "gpt_41_mini_results.csv"

# The list of destinations we want directions to:
DESTINATIONS = [
    "703 off cs",
    "bathroom (700 fl pubr-f bsvc)",
    "750E3 RLAB EE",
    "720 off cs"
]


def encode_image_to_data_url(image_path: str) -> str:
    """
    Reads an image from disk, base64-encodes it, and returns a data:URL string.
    """
    with open(image_path, "rb") as f:
        raw = f.read()
    b64 = base64.b64encode(raw).decode("utf-8")
    ext = os.path.splitext(image_path)[1].lower()
    if ext in [".jpg", ".jpeg"]:
        mime = "image/jpeg"
    elif ext == ".png":
        mime = "image/png"
    else:
        raise RuntimeError(f"Unsupported image extension: {ext}")
    return f"data:{mime};base64,{b64}"


# 3. Load the prompt template (with bracketed placeholders):
with open(PROMPT_FILE, "r", encoding="utf-8") as f:
    template = f.read()

# 4. Load the node metadata JSON (as a raw string):
with open(NODE_METADATA_FILE, "r", encoding="utf-8") as f:
    node_metadata_json_str = f.read().strip()

# 5. Base64‐encode the floor‐plan once:
floorplan_data_url = encode_image_to_data_url(FLOORPLAN_IMAGE_PATH)

# 6. Prepare our CSV file and write header:
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(
        csvfile,
        fieldnames=[
            "query_image_name",
            "destination",
            "identified_location",
            "approximate_grid_position",
            "facing_direction",
            "reasoning",
            "fastest_route",
            "full_model_answer"
        ]
    )
    writer.writeheader()

    # 7. Iterate over every image in the query folder:
    image_paths = glob.glob(os.path.join(QUERY_IMAGES_FOLDER, "*"))
    print(f"Found {len(image_paths)} images in {QUERY_IMAGES_FOLDER}.")
    for i,img_path in enumerate(image_paths):
        # Only process common image extensions
        ext = os.path.splitext(img_path)[1].lower()
        if ext not in [".jpg", ".jpeg", ".png"]:
            continue

        query_image_name = os.path.basename(img_path)
        query_data_url    = encode_image_to_data_url(img_path)

        # 8. For each fixed destination, build a distinct prompt and call the API:
        for j,destination in enumerate(DESTINATIONS):
            # Split the template at the three placeholders in order:
            part_before_floor, remainder               = template.split("[FLOOR_PLAN_IMAGE]", 1)
            part_before_metadata, remainder            = remainder.split("[NODE_METADATA_JSON]", 1)
            part_before_query, part_after_query        = remainder.split("[QUERY_IMAGE]", 1)

            prompt_text_1 = part_before_floor
            prompt_text_2 = part_before_metadata
            prompt_text_3 = part_before_query
            prompt_text_4 = part_after_query.replace("[DESTINATION]", destination)

            # Build the “chat” payload as a list of text blocks and image blocks:
            messages = [
                # (1) Text up to the floor‐plan placeholder
                {"role": "user", "content": prompt_text_1},

                # (2) The floor‐plan image
                {"role": "user", "content": [
                    {"type": "image_url", "image_url": {"url": floorplan_data_url}}
                ]},

                # (3) Text between floorplan and metadata
                {"role": "user", "content": prompt_text_2},

                # (4) The raw node‐metadata JSON
                {"role": "user", "content": node_metadata_json_str},

                # (5) Text between metadata and query image
                {"role": "user", "content": prompt_text_3},

                # (6) The query image itself
                {"role": "user", "content": [
                    {"type": "image_url", "image_url": {"url": query_data_url}}
                ]},

                # (7) The remaining text with [DESTINATION] replaced
                {"role": "user", "content": prompt_text_4},
            ]

            # 8b. Call the new client API for chat completions:
            try:
                resp = client.chat.completions.create(
                    model=API_MODEL,
                    messages=messages
                )
            except Exception as e:
                print(f"ERROR: {query_image_name} → {destination}: {e}")
                continue

            # 8c. Extract the assistant’s reply:
            raw_reply = resp.choices[0].message.content

            # 8d. Try to locate the final JSON object in raw_reply:
            identified_location       = ""
            approximate_grid_position = ""
            facing_direction          = ""
            reasoning_text            = ""
            fastest_route_text        = ""
            parsed_full_answer        = ""

            try:
                last_open  = raw_reply.rfind("{")
                last_close = raw_reply.rfind("}")
                json_str   = raw_reply[last_open:last_close+1]
                parsed     = json.loads(json_str)
                identified_location       = parsed.get("identified_location", "")
                approximate_grid_position = parsed.get("approximate_grid_position", "")
                facing_direction          = parsed.get("facing_direction", "")
                reasoning_text            = parsed.get("reasoning", "")
                fastest_route_text        = parsed.get("fastest_route", "")
                parsed_full_answer        = parsed.get("full_answer", "")
            except Exception:
                parsed_full_answer = raw_reply
                reasoning_text    = ""
                print(f"⚠️ Warning: could not parse JSON from response for {query_image_name} → {destination}. "
                      "Storing raw reply as full_answer.")
            to_write = {
                "query_image_name":           query_image_name,
                "destination":                destination,
                "identified_location":        identified_location,
                "approximate_grid_position":  approximate_grid_position,
                "facing_direction":           facing_direction,
                "reasoning":                  reasoning_text,
                "fastest_route":              fastest_route_text,
                "full_model_answer":          parsed_full_answer
            }
            # 9. Write one row into CSV for this (image, destination) pair:
            writer.writerow(to_write)

            print(f"✔ Processed {query_image_name} → {destination}")
            print(to_write) 

print(f"\nAll done. Results written to {OUTPUT_CSV}.")


Found 11 images in CEPSR - Floor 7/Query images.
✔ Processed location1_east.jpeg → 703 off cs
{'query_image_name': 'location1_east.jpeg', 'destination': '703 off cs', 'identified_location': 'Node 2 (facing west)', 'approximate_grid_position': '', 'facing_direction': 'West', 'reasoning': '1. Floor Pattern Check: The floor in the photo shows a black-and-white checker tile pattern that matches many nodes on this floor, including node 2 west. This pattern is consistent throughout most corridors and rooms and is a strong general marker here.\n2. Ceiling & Lighting: The ceiling has white fluorescent ceiling panels and a distinctive purple accent strip running along the ceiling close to the right wall. This purple accent strip feature appears prominently in the descriptions of nodes on the west-facing corridors, particularly node 2 west, which mentions purple accent strip along the ceiling.\n3. Wall/Signage/Doors: On the right side of the photo, there are gray doors that look quite plain, sim

In [ ]:
# 16k tokens for prompt only